In [1]:
from NCA import *
import arc_agi_utils as aau
import cv2
import vft
import os
import matplotlib.pyplot as plt
import numpy as np

from utils import make_path

# Make path for videos

In [2]:
path_video = "Saved_frames"
make_path(path_video)

Path: Saved_frames already exists, all OK!


# Video making functions

In [3]:
def write_frame(x, path, frame_number, height, width, chn):
    image_np = x.clone().detach().cpu().permute(0,3,2,1).numpy().clip(0,1)[0,:,:,:3]
    plt.imsave(f"{path}/frame_{frame_number}.png", image_np)

def make_video(path, total_frames, height, width, vid_num = "r"):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(path+'/' +vid_num+'.mp4', fourcc, 15.0, (height, width))
    for frame_number in range(total_frames):
       frame_path = path+f"/frame_{frame_number}.png"
       frame = cv2.imread(frame_path)
       #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
       frame = cv2.flip(frame,1)
       frame = cv2.rotate(frame,cv2.ROTATE_90_COUNTERCLOCKWISE)
       frame = cv2.resize(frame, (height, width), interpolation=cv2.INTER_NEAREST)

       out.write(frame)
    out.release()

# CA settings

In [4]:
DEVICE = vft.DEVICE
CHANNELS = vft.CHANNELS
BATCH_SIZE = vft.BATCH_SIZE
MASKING = vft.MASKING
GENESIZE = vft.GENESIZE

# Load ARC-AGI

In [5]:
training_path = "ArcData/data/training"
eval_path = "ArcData/data/evaluation"
(inputs, outputs), (eval_in, eval_out)= aau.import_data(training_path)
inputs, outputs, eval_in, eval_out = aau.filter_problems(inputs, outputs, eval_in, eval_out)
max_train = aau.max_n_colors(inputs, outputs)
max_eval = aau.max_n_colors(eval_in, eval_out)
max_colors = max(max_train, max_eval) +1

# Problems to test per model (you can choose your own)

In [6]:
v3 = {0, 7, 12, 13, 140, 24, 154, 155, 159, 162, 41, 169, 49, 185, 63, 191, 69, 71, 199, 205, 209, 83, 213, 89, 94, 222, 223, 98, 103, 232, 107, 235, 118, 254}
v4 = {7, 12, 140, 24, 154, 155, 159, 162, 41, 169, 172, 49, 185, 69, 205, 208, 209, 213, 89, 94, 222, 103, 107, 116, 118, 120, 249}
v1 = {191, 258, 3, 103, 169, 208, 81, 209, 116, 213, 150, 254, 24, 185, 154, 222, 159}
ca = {7, 136, 140, 142, 24, 159, 162, 41, 169, 172, 49, 185, 63, 191, 69, 71, 205, 209, 213, 89, 94, 223, 103, 231, 235, 118, 120, 254}
test = {0}

# Make Videos 

In [7]:
for problem in test:


    mode = "rgb"
    genes = [i for i in range(GENESIZE)]
    nca_in = [aau.arc_to_nca_space(max_colors, ip, CHANNELS,GENESIZE, mode=mode, gene_location=genes) for ip in inputs[problem]]
    nca_out = [aau.arc_to_nca_space(max_colors, ip, CHANNELS,GENESIZE, mode=mode, gene_location=genes) for ip in outputs[problem]]

    eval_nca_in = [aau.arc_to_nca_space(max_colors, ip, CHANNELS,GENESIZE, mode=mode, gene_location=genes) for ip in eval_in[problem]]
    eval_nca_out = [aau.arc_to_nca_space(max_colors, ip, CHANNELS,GENESIZE, mode=mode, gene_location=genes) for ip in eval_out[problem]]



    with torch.no_grad():

        nca = CA(CHANNELS, vft.GENE_HIDDEN_N +vft.GENE_PROP_HIDDEN_N)
        nca.load_state_dict(torch.load(f"TrainedARCModels/{type(nca).__name__}/problem_" +str(problem)+".pth"))
        nca = nca.to(DEVICE)

    nca.eval()


    image_np = aau.nca_to_rgb_image(eval_nca_in[0])[:,:,:4]*255
    image_np = cv2.resize(image_np, (10 * image_np.shape[1], 10 * image_np.shape[0]), interpolation=cv2.INTER_NEAREST)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    cv2.imwrite("Saved_frames/Saved_start_and_end/"  + type(nca).__name__ +"problem_start_" + str(problem)+".jpg", image_np)

    image_np = aau.nca_to_rgb_image(eval_nca_out[0])[:, :, :4] * 255
    image_np = cv2.resize(image_np, (10 * image_np.shape[1], 10 * image_np.shape[0]), interpolation=cv2.INTER_NEAREST)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    cv2.imwrite("Saved_frames/Saved_start_and_end/" + type(nca).__name__ + "problem_end_" + str(problem) + ".jpg",
                image_np)


    x = eval_nca_in[0].tile(1,1,1,1)

    for i in range(128):
        x = nca(x, 0.5)
        x = x.detach()
        write_frame(x, path_video, i, 10*x.shape[3],10*x.shape[2], CHANNELS)

    make_video(path_video, 128, 10*x.shape[3],10*x.shape[2], type(nca).__name__ +"problem_" + str(problem))

/tmp/ipykernel_24633/877401437.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  nca.load_state_dict(torch.load(f"TrainedARCModels/{type(nca).__name__}/problem_" +str(pro